Author
vokrut--42Robotics

In [0]:
#Import required frameworks

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow.python.data import Dataset
from pandas.io.json import json_normalize
import json


from google.colab import drive
drive.mount('/content/drive')

#upload json data file 
quantum_data = pd.read_json("drive/My Drive/qc/01/3125_random_5_qubit_circuits.json",
                            orient='records', lines=True, precise_float="precise_float")

#quantum_data.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Function takes the raw data as a .json file, split the data into two parts, 
#the first 32 columns represent state vectors the last 173 colums represent
#the parameters of each gate.
#Parameters: quantum_data - the raw data read from .json file
#             gate_num - current gate number from the range from  1 to 40
#Returns: a new pandas dataframe with separated values of the statevectors and gate's parameters

def gate_data_preproc(quantum_data, gate_num):
  #since we care only about the value of the angles in U3 Gate, check for this type of gate
  #if the gate is not a U3 gate, set the angle values to 0
  is_not_u3gate = lambda data: data["Gate_Type"] != 'U3Gate'
  norm_data = json_normalize(quantum_data)
  dataframe = pd.DataFrame()  #create new data table

  #.loc[] - accesses a group of rows and columns by label(s) or a boolean array.
  norm_data.loc[(is_not_u3gate(norm_data), 'Angle_1')] = 0.0
  norm_data.loc[(is_not_u3gate(norm_data), 'Angle_2')] = 0.0
  norm_data.loc[(is_not_u3gate(norm_data), 'Angle_3')] = 0.0

  #iterate through each subcateggory, pass if GateType or GateNumber
  #store in the new table 
  for key in norm_data:
    # print(key)
    if key == 'Gate_Number' or key == 'Gate_Type':
      continue
    dataframe[key+gate_num] = norm_data[key]
    if key in ['Target', 'Control']:
      dataframe[key+gate_num] += 1
      # print(dataframe[key+gate_num])

  return dataframe

norm_qdata = json_normalize(quantum_data['statevector'])
# print(qdata.head())
#iterate in data set. where key represent a gate number on each iteration
for key in quantum_data:
  # print(key)
  if "gate" in key:                   #if key == gate, store the gate number
    gate_num = "_" + str(key[-2:])
    gate_dataframe = gate_data_preproc(quantum_data[key], gate_num)
    norm_qdata = norm_qdata.join(gate_dataframe)

#randomize data
# norm_qdata = norm_qdata.reindex(
#     np.random.permutation(norm_qdata.index))
norm_qdata
# display.display(norm_qdata)

,state_11111,state_10111,state_11011,state_11100,state_01001,state_01100,state_00000,state_00100,state_01000,state_11110,state_11000,state_00001,state_00111,state_00010,state_11010,state_10000,state_10110,state_10010,state_00101,state_10100,state_01111,state_11001,state_10001,state_00011,state_10101,state_01010,state_11101,state_10011,state_01110,state_01101,state_00110,state_01011,Angle_3_11,Control_11,Angle_1_11,Target_11,Angle_2_11,Angle_3_30,Control_30,Angle_2_30,...,Angle_3_14,Control_14,Angle_1_14,Target_14,Angle_2_14,Angle_3_26,Control_26,Angle_1_26,Target_26,Angle_2_26,Angle_3_07,Control_07,Angle_1_07,Target_07,Angle_2_07,Angle_3_03,Control_03,Angle_1_03,Target_03,Angle_2_03,Angle_3_21,Control_21,Angle_1_21,Target_21,Angle_2_21,Angle_3_22,Control_22,Angle_1_22,Target_22,Angle_2_22,Angle_3_36,Control_36,Angle_2_36,Target_36,Angle_1_36,Angle_3_04,Control_04,Angle_1_04,Target_04,Angle_2_04
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1024,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,1,0.0,2,0.0,0.000000,0,0.000000,...,0.709553,4,0.0,0,0.0,0.000000,2,0.000000,0,0.0,0.0000,2,0.000000,1,0.000000,0.713632,3,0.0,0,0.0,-0.709553,3,-0.804929,0,0.0,0.000000,4,0.000000,3,0.0,0.0,0,0.0,0,0.0,1.00000,4,0.500000,0,0.0
1,0,0,0,1,0,7,0,6,0,0,0,0,0,0,0,0,0,0,276,0,0,0,0,0,70,0,126,0,0,538,0,0,1.000000,3,1.0,0,0.0,0.500000,5,-0.500000,...,1.088594,2,0.0,0,0.0,0.500000,4,1.392736,0,-0.5,0.0000,4,0.000000,3,0.000000,0.779094,1,0.0,0,0.0,0.000000,2,0.000000,1,0.0,-1.109076,1,-0.735356,0,0.0,0.0,0,0.0,0,0.0,0.30387,4,0.000000,0,0.0
2,0,0,0,0,473,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,507,0,0,0,0,0,0,0,44,-0.584628,3,0.0,0,0.0,0.500000,5,0.000000,...,1.000000,4,1.0,0,0.0,-0.500000,3,0.000000,0,0.0,0.8808,2,0.000000,0,0.000000,0.000000,1,0.0,2,0.0,1.244226,5,0.000000,0,0.0,0.000000,5,0.000000,3,0.0,0.0,5,0.0,0,0.0,-0.88080,2,0.000000,0,0.0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1023,0,0.102862,5,0.0,0,0.0,0.000000,0,0.000000,...,0.000000,3,0.0,5,0.0,0.000000,5,0.000000,0,0.0,1.0000,5,0.500000,0,0.000000,0.368304,3,0.0,0,0.0,0.000000,1,0.000000,5,0.0,0.000000,1,0.000000,0,0.0,0.0,0,0.0,0,0.0,0.00000,4,0.000000,3,0.0
4,0,0,0,0,0,0,1024,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,3,0.0,5,0.0,1.950577,5,0.000000,...,1.000000,5,0.5,0,0.0,0.000000,1,0.000000,5,0.0,1.0000,4,0.500000,0,0.000000,0.984549,3,0.0,0,0.0,-0.500000,5,0.000000,0,0.0,0.531491,5,0.000000,0,0.0,0.0,5,0.0,0,0.0,0.00000,4,0.000000,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3120,0,0,0,0,0,0,565,0,0,0,0,459,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.500000,1,0.0,0,0.0,0.000000,0,0.000000,...,1.000000,5,0.5,0,0.0,0.000000,0,0.000000,0,0.0,0.0000,4,0.000000,2,0.000000,0.066918,4,0.0,0,0.0,0.000000,2,0.000000,0,0.0,0.000000,3,0.000000,0,0.0,0.0,0,0.0,0,0.0,0.00000,3,0.000000,4,0.0
3121,0,0,0,0,0,0,0,0,0,0,0,0,0,394,0,0,0,93,0,0,0,0,0,451,0,0,0,86,0,0,0,0,0.000000,3,0.0,1,0.0,0.984881,5,1.320586,...,-0.500000,3,0.0,0,0.0,-0.175874,1,0.000000,0,0.0,-0.5000,4,0.000000,0,0.000000,-0.267541,1,0.0,0,0.0,0.635573,2,0.000000,0,0.0,0.000000,4,0.000000,2,0.0,0.0,0,0.0,0,0.0,0.00000,3,0.000000,1,0.0
3122,0,0,0,0,0,134,0,890,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.845056,1,0.0,0,0.0,0.000000,0,0.000000,...,0.000000,3,0.0,1,0.0,0.000000,4,0.000000,0,0.0,0.0000,3,0.767423,0,1.947449,1.545911,4,0.0,0,0.0,0.000000,2,0.000000,5,0.0,0.500000,5,0.000000,0,0.0,0.0,0,0.0,0,0.0,0.00000,4,0.000000,3,0.0
3123,328,168,0,0,17,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,6,162,0,326,0,0,0,0,15,0.500000,5,0.0,0,0.0,0.000000,0,0.000000,...,0.000000,3,0.0,0,0.0,0.000000,0,0.000000,0,0.0,0.0000,5,0.000000,2,0.000000,1.000000,4,0.0,0,0.0,0.000000,0,0.000000,0,0.0,0.000000,0,0.000000,0,0.0,0.0,0,0.0,0,0.0,0.00000,4

In [0]:
#analize features
print(norm_qdata.columns)

Index(['state_11111', 'state_10111', 'state_11011', 'state_11100',
       'state_01001', 'state_01100', 'state_00000', 'state_00100',
       'state_01000', 'state_11110',
       ...
       'Angle_3_36', 'Control_36', 'Angle_2_36', 'Target_36', 'Angle_1_36',
       'Angle_3_04', 'Control_04', 'Angle_1_04', 'Target_04', 'Angle_2_04'],
      dtype='object', length=237)


In [0]:
#describe data
norm_qdata.describe()

,state_11111,state_10111,state_11011,state_11100,state_01001,state_01100,state_00000,state_00100,state_01000,state_11110,state_11000,state_00001,state_00111,state_00010,state_11010,state_10000,state_10110,state_10010,state_00101,state_10100,state_01111,state_11001,state_10001,state_00011,state_10101,state_01010,state_11101,state_10011,state_01110,state_01101,state_00110,state_01011,Angle_3_11,Control_11,Angle_1_11,Target_11,Angle_2_11,Angle_3_30,Control_30,Angle_2_30,...,Angle_3_14,Control_14,Angle_1_14,Target_14,Angle_2_14,Angle_3_26,Control_26,Angle_1_26,Target_26,Angle_2_26,Angle_3_07,Control_07,Angle_1_07,Target_07,Angle_2_07,Angle_3_03,Control_03,Angle_1_03,Target_03,Angle_2_03,Angle_3_21,Control_21,Angle_1_21,Target_21,Angle_2_21,Angle_3_22,Control_22,Angle_1_22,Target_22,Angle_2_22,Angle_3_36,Control_36,Angle_2_36,Target_36,Angle_1_36,Angle_3_04,Control_04,Angle_1_04,Target_04,Angle_2_04
count,3125.00000,3125.00000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.00000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,...,3125.000000,3125.0000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.000000,3125.0,3125.000000,3125.0,3125.0,3125.0,3125.000000,3125.00000,3125.000000,3125.000000,3125.000000
mean,4.31136,6.48704,5.440320,10.389760,21.904640,21.527680,301.729600,70.645440,70.895360,4.796160,21.183360,75.218560,9.818240,71.248000,11.659520,73.753280,9.896320,24.461440,20.590720,23.15904,6.001600,8.175040,22.288640,23.355200,10.566720,24.221120,5.729920,12.576640,10.856000,9.777280,20.718720,10.617280,0.160400,2.869760,0.088459,0.782400,0.037538,0.019997,0.725440,0.007079,...,0.136053,2.4960,0.078139,0.659520,0.035556,0.033170,1.118400,0.026638,0.224320,0.016599,0.198562,2.946560,0.118504,0.882560,0.047618,0.277315,2.639040,0.134090,0.850880,0.043425,0.076432,1.684800,0.053384,0.435520,0.023289,0.060938,1.618880,0.044570,0.356480,0.022323,0.0,0.248320,0.0,0.0,0.0,0.261983,2.81888,0.135210,0.959360,0.032736
std,33.85683,45.55274,40.711141,63.811295,103.448676,100.585487,381.365618,198.916668,196.640008,34.982203,98.611662,206.546912,57.986175,199.348666,70.748684,202.736519,61.489957,103.800086,94.140423,100.42303,49.221113,54.362371,98.093931,103.609072,64.610409,103.422059,45.067115,67.201416,65.580539,64.113433,100.754636,63.032506,0.480760,1.666111,0.282246,1.536583,0.237262,0.175556,1.566743,0.094996,...,0.446126,1.8827,0.263402,1.464227,0.226041,0.238508,1.783782,0.157961,0.929278,0.161958,0.527471,1.313292,0.330729,1.572806,0.258365,0.592346,1.116005,0.360369,1.384997,0.247456,0.348591,1.960602,0.225969,1.260584,0.182924,0.318370,1.987886,0.207012,1.123729,0.177045,0.0,1.038827,0.0,0.0,0.0,0.546596,1.21365,0.346765,1.500943,0.226075
min,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.867883,0.000000,-0.983906,0.000000,-0.500000,-1.700741,0.000000,-0.500000,...,-1.734532,0.0000,-0.998146,0.000000,-0.500000,-1.540465,0.000000,-0.871231,0.000000,-0.500000,-1.908105,1.000000,-0.997631,0.000000,-0.500000,-1.763002,1.000000,-0.999282,0.000000,-0.500000,-1.627857,0.000000,-0.940143,0.000000,-0.500000,-1.363557,0.000000,-0.951870,0.000000,-0.500000,0.0,0.000000,0.0,0.

In [0]:
#Define and store the input features
#Parameters: normalized dataframe
#Returns: features, e.g. last 173 colums of the dataframe

def preprocess_features(quantum_data):
  #define features
  selected_features = quantum_data.filter(regex='^(?!state)', axis=1)
  return selected_features

#Define and store the target
#Parameters: normalized dataframe
#Returns: targets, e.g. first 32 colums of the dataframe
def preprocess_targets(quantum_data):
  output_targets = pd.DataFrame()
  output_targets = quantum_data.filter(regex='^state', axis=1) 
  output_targets /= 1024
  return output_targets

def preprocess_feature_labels(quantum_data):
  feature = reprocess_features(quantum_data)
  labels = reprocess_targets(quantum_data)
  return (feature, labels)

#display to male sure you are thinking correct
#display.display(preprocess_features(norm_qdata))
#display.display(preprocess_targets(norm_qdata))


In [0]:
#Set training data.
training_feature = preprocess_features(norm_qdata.head(2185))
training_target = preprocess_targets(norm_qdata.head(2185))

#Set valiation data.
validating_feature = preprocess_features(norm_qdata.tail(100))
validating_target = preprocess_targets(norm_qdata.tail(100))


#displays features and targets
# print("Training feature")
# display.display(training_feature)

# print("Training targets")
# display.display(training_target)

# print("Validating data")
# display.display(validating_feature)

In [0]:
#Construct the TensorFlow Feature Columns.
#Parameters: input_features: The names of the numerical input features to use.
#Returns: A set of feature columns

def construct_feature_columns(input_features):
  return set([tf.feature_column.numeric_column(my_feature)
              for my_feature in input_features])

In [0]:
# A custom input_fn for sending qauntum data to the estimator for training.
#   Parameters:
#     features: The training features.
#     labels: The training labels.
#     batch_size: Batch size to use during training.
#     num_epochs: number of epochs
#   Returns:
#     A function that returns batches of training features and labels during
#     training.


def create_training_input_fn(features, labels, batch_size, num_epochs=None, shuffle=True):
  def _input_fn(num_epochs=None, shuffle=True):
    # Input pipelines are reset with each call to .train(). To ensure model
    # gets a good sampling of data, even when number of steps is small, we 
    # shuffle all the data before creating the Dataset object

    raw_features = {key:np.array(value) for key, value in dict(features).items()}
    # print(raw_features)
    raw_targets = np.array(labels)
   
    ds = Dataset.from_tensor_slices((raw_features,raw_targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    if shuffle:
      ds = ds.shuffle(10000)
    
    # Return the next batch of data.
    feature_batch, label_batch = ds.make_one_shot_iterator().get_next()
    return feature_batch, label_batch

  return _input_fn

In [0]:
# A custom input function for sending qauntum data to the estimator for predictions.
#   Parameters:
#     features: The features to base predictions on.
#     labels: The labels of the prediction examples.
#     batch_size: size of the taken batch
#   Returns:
#     features and labels for predictions.

def create_predict_input_fn(features, targets, batch_size):
  def _input_fn():
    raw_features = {key:np.array(value) for key, value in dict(features).items()}
    # print(raw_features)
    raw_targets = np.array(targets)
    
    ds = Dataset.from_tensor_slices((raw_features, raw_targets)) # warning: 2GB limit
    ds = ds.batch(batch_size)
    
        
    # Return the next batch of data.
    feature_batch, label_batch = ds.make_one_shot_iterator().get_next()
    return feature_batch, label_batch

  return _input_fn

In [0]:
# Trains a neural network regression model for the qauntum dataset.
#   In addition to training, this function also prints training progress information,
#   a plot of the training and validation loss over time.
#   Parameters:
#     learning_rate: A `float`, the learning rate to use.
#     steps: A non-zero `int`, the total number of training steps. A training step
#            consists of a forward and backward pass using a single batch.
#     batch_size: A non-zero `int`, the batch size.
#     hidden_units: A `list` of int values, specifying the number of neurons in each layer.
#     training_examples: A `DataFrame` containing the training features.
#     training_targets: A `DataFrame` containing the training labels.
#     validation_examples: A `DataFrame` containing the validation features.
#     validation_targets: A `DataFrame` containing the validation labels.
#   Returns:
#     The trained `DNNRegresion` object.

def train_nn_classification_model(
    learning_rate,
    steps,
    batch_size,
    hidden_units,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):

  periods = 10
  # Caution: input pipelines are reset with each call to train. 
  # If the number of steps is small, your model may never see most of the data.  
  # So with multiple `.train` calls like this you may want to control the length 
  # of training with num_epochs passed to the input_fn. Or, you can do a really-big shuffle, 
  # or since it's in-memory data, shuffle all the data in the `input_fn`.
  steps_per_period = steps / periods  
  
  # Create the input functions.
  predict_training_input_fn = create_predict_input_fn(
    training_examples, training_targets, batch_size)
  
  predict_validation_input_fn = create_predict_input_fn(
    validation_examples, validation_targets, batch_size)
  
  #training input function
  training_input_fn = create_training_input_fn(
    training_examples, training_targets, batch_size)
  
  # Create feature columns using customs function
  feature_columns = construct_feature_columns(training_examples)
  #set the optimiser
  my_optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)



  #   hidden_units: Iterable of number hidden units per layer.
  #                 All layers are fully connected. Ex. [64, 32] means first
  #                 layer has 64 nodes and second one has 32.
  # feature_columns: An iterable containing all the feature columns used by the model.
  #                  All items in the set should be instances of classes derived from _FeatureColumn.
  # model_dir: Directory to save model parameters, graph and etc.
  #            This can also be used to load checkpoints from the directory into a estimator to continue training a previously saved model.
  # label_dimension: Number of regression targets per example.
  #                 This is the size of the last dimension of the labels and logits Tensor objects (typically, these have shape [batch_size, label_dimension]).
  # weight_column: A string or a _NumericColumn created by tf.feature_column.numeric_column
  #               defining feature column representing weights. It is used to down weight or boost examples during training.
  #               It will be multiplied by the loss of the example. If it is a string, it is used as a key to fetch weight tensor from the features.
  #               If it is a _NumericColumn, raw tensor is fetched by key weight_column.key, then weight_column.normalizer_fn is applied on it to get weight tensor.
  # optimizer: An instance of tf.keras.optimizers.Optimizer used to train the model.
  #            Can also be a string (one of 'Adagrad', 'Adam', 'Ftrl', 'RMSProp', 'SGD'), or callable. Defaults to Adagrad optimizer.
  # activation_fn: Activation function applied to each layer. If None, will use tf.nn.relu.
  # dropout: When not None, the probability we will drop out a given coordinate.
  # config: RunConfig object to configure the runtime settings.
  # warm_start_from: A string filepath to a checkpoint to warm-start from, or a WarmStartSettings object
  #                   to fully configure warm-starting. If the string filepath is provided instead of a WarmStartSettings,
  #                   then all weights are warm-started, and it is assumed that vocabularies and Tensor names are unchanged.
  # loss_reduction: One of tf.losses.Reduction except NONE. Describes how to reduce training loss over batch. Defaults to SUM_OVER_BATCH_SIZE.
  # batch_norm: Whether to use batch normalization after each hidden layer.

  # Create a DNNRegression object.
  # Use .train(trainig_data) to train the model
  # use .predict(predict_training_function) to make a prediction using a model
  classifier = tf.estimator.DNNEstimator(
      head=tf.contrib.estimator.multi_label_head(n_classes=32),
      hidden_units=hidden_units,
      feature_columns=feature_columns,
      optimizer=my_optimizer,
      config=tf.contrib.learn.RunConfig(keep_checkpoint_max=1)
  )

  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print("Training model...")
  print("LogLoss error (on validation data):")
  training_errors = []
  validation_errors = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    print("Period = ", period)
    classifier.train(
        input_fn=training_input_fn,
        steps=steps_per_period
    )
  
    # Take a break and compute predictions.
  #   training_predictions = list(classifier.predict(input_fn=predict_training_input_fn))
  #   # print(training_predictions)
  #   training_predictions = np.array([item['probabilities'] for item in training_predictions])
  #   error_on_training = tf.keras.losses.KLDivergence(training_predictions, training_targets)

  #   # error_on_training = math.sqrt(error_on_training)
        
  #   validation_predictions = list(classifier.predict(input_fn=predict_validation_input_fn))
  #   validation_predictions = np.array([item['probabilities'] for item in validation_predictions])     
  #   error_on_validating = tf.keras.losses.KLDivergence(validation_predictions, validation_targets)

  #   # error_on_validating = math.sqrt(error_on_validating)

  #   #WIP on errors and metrics!!!!
  #   # Occasionally print the current loss.
    
  #   print(error_on_validating)
  #   # Add the loss metrics from this period to our list. ??to plot the graph later??
  #   training_errors.append(error_on_training)
  #   validation_errors.append(error_on_validating)
  # print("Model training finished.")
  
  # # Calculate final predictions (not probabilities, as above).
  # # final_predictions = classifier.predict(input_fn=predict_validation_input_fn)
  # # final_predictions = np.array([item['class_ids'][0] for item in final_predictions])
  
  
  # # accuracy = metrics.accuracy_score(validation_targets, final_predictions)
  # # print("Final accuracy (on validation data): %0.2f" % accuracy)

  # # Output a graph of loss metrics over periods.
  # plt.ylabel("MSE")
  # plt.xlabel("Periods")
  # plt.title("MSE vs. Periods")
  # plt.plot(training_errors, label="training")
  # plt.plot(validation_errors, label="validation")
  # plt.legend()
  # plt.show()
  
  return classifier

In [49]:
model = train_nn_classification_model(
    learning_rate=0.0001,
    steps=1000,
    batch_size=32,
    hidden_units=[1024, 512, 256],
    training_examples=training_feature,
    training_targets=training_target,
    validation_examples=validating_feature,
    validation_targets=validating_target)


INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5984085c18>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpy_vievyp', '_session_creation_timeout_secs': 7200}
Training model...
LogLoss error (on validation data):
Period =  0
INFO:tensorflow:

KeyboardInterrupt: ignored

In [0]:
def my_input_fn(features, targets, batch_size=1,  shuffle=True, num_epochs=None):
  features = {key:np.array(value) for key, value in dict(features).items()}
  #  print(raw_features)
  # raw_targets = np.array(labels)
   
  ds = Dataset.from_tensor_slices((features, targets)) # warning: 2GB limit
  ds = ds.batch(batch_size).repeat(num_epochs)
    
  if shuffle:
    ds = ds.shuffle(10000)
    
  # Return the next batch of data.
  features, labels = ds.make_one_shot_iterator().get_next()  
  return features, labels

In [0]:
#Evaluate the model

quantum_test_data = pd.read_json("drive/My Drive/qc/02/3125_random_5_qubit_circuits.json",
                            orient='records', lines=True, precise_float="precise_float")

norm_test_qdata = json_normalize(quantum_test_data['statevector'])
#iterate in data set. where key represent a gate number on each iteration
for key in quantum_test_data:
  # print(key)
  if "gate" in key:                   #if key == gate, store the gate number
    gate_num = "_" + str(key[-2:])
    gate_dataframe = gate_data_preproc(quantum_test_data[key], gate_num)
    norm_test_qdata = norm_test_qdata.join(gate_dataframe)

#randomize data
# norm_qdata = norm_qdata.reindex(
#     np.random.permutation(norm_qdata.index))
norm_test_qdata

test_examples = preprocess_features(norm_test_qdata.head(100))
test_targets = preprocess_targets(norm_test_qdata.head(100))

predict_testing_input_fn = lambda: my_input_fn(test_examples, 
                                               test_targets, 
                                               num_epochs=1, 
                                               shuffle=False)

test_predictions = model.predict(input_fn=predict_testing_input_fn)
test_predictions = np.array([item['probabilities'] for item in test_predictions]) 

root_mean_squared_error = math.sqrt(
    metrics.mean_squared_error(test_predictions, test_targets))

print("Final RMSE (on test data): %0.2f" % root_mean_squared_error)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpa1jukx0i/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Final RMSE (on test data): 0.39


In [0]:
#Predict the model
quantum_test_data = pd.read_json("drive/My Drive/qc/02/3125_random_5_qubit_circuits.json",
                            orient='records', lines=True, precise_float="precise_float")

norm_test_qdata = json_normalize(quantum_test_data['statevector'])
#iterate in data set. where key represent a gate number on each iteration
for key in quantum_test_data:
  # print(key)
  if "gate" in key:                   #if key == gate, store the gate number
    gate_num = "_" + str(key[-2:])
    gate_dataframe = gate_data_preproc(quantum_test_data[key], gate_num)
    norm_test_qdata = norm_test_qdata.join(gate_dataframe)

#randomize data
# norm_qdata = norm_qdata.reindex(
#     np.random.permutation(norm_qdata.index))
norm_test_qdata

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predict_features = preprocess_features(norm_test_qdata.tail(1))
actual_targets = preprocess_targets(norm_test_qdata.tail(1))

predict1_fn = lambda: input_fn(predict_features, batch_size = 256)

predictions = model.predict(input_fn=predict1_fn)

for predict, expect in zip(predictions, actual_targets):
  # print(predict)
  # print(expect)
  class_id = predict['probabilities']
  probability = predict['probabilities'][class_id]
  print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(SPECIES[class_id], 100 * probability, expect))

test_predictions = np.array([item['probabilities'] for item in test_predictions]) 

root_mean_squared_error = math.sqrt(
    metrics.mean_squared_error(test_predictions, test_targets))

print("Final RMSE (on test data): %0.2f" % root_mean_squared_error)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmppoclavb2/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([-0.10051182, -0.08068331,  0.10833137,  0.12927678,  0.18622164,
        0.17171247,  0.05263708, -0.02989437, -0.07431062,  0.06395443,
        0.10532874,  0.15675855,  0.04360103,  0.07780567, -0.06391643,
        0.1886475 , -0.09269717, -0.00131911, -0.03846923,  0.10475976,
       -0.03630986, -0.13838701,  0.14469472,  0.0992074 ,  0.00906586,
        0.20825139,  0.05467982, -0.27829605,  0.01115839, -0.02639122,
        0.1597413 ,  0.30464152], dtype=float32)}
state_00010


KeyError: ignored